# **5장. 합성곱 신경망 1**

**5.1.2 합성곱 신경망 구조**

: **입력층**에서 입력받은 데이터가 **합성곱층**과 **풀링층**을 거치면서 입력 이미지의 주요 특성 벡터가 추출되고, 추출된 주요 특성 벡터들은 **완전연결층**을 거치며 1차원 벡터로 변환되며, **출력층**에서 활성화함수인 소프트멕스 함수를 사용하여 최종 결과가 출력됨

**5.1.3 1D, 2D, 3D 합성곱**

1. 1D 합성곱

